## Import Libraries

In [1]:
from pymongo import MongoClient
import json
from bson import ObjectId
import pandas as pd
from bson import json_util, ObjectId

In [2]:
import pprint

In [3]:
MONGODB_HOST = 'localhost'
MONGODB_PORT = 27017
DBS_NAME = 'mydb'
COLLECTION_NAME = 'projects'
FIELDS = "FeatureCollection"

In [ ]:
def dashboard_projects():
    connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
    collection = connection[DBS_NAME][COLLECTION_NAME]
    projects = collection.find_one({"type": "FeatureCollection"}, {"_id":0})

    return projects

In [ ]:
dashboard_projects()

In [ ]:
# Call all documents
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
projects2 = collection.find_one({"type": "FeatureCollection"}, {"_id":0})

In [8]:
client = MongoClient()
db = client.mydb
collection = db.projects

In [ ]:
from bson.son import SON
from pymongo import MongoClient

db = MongoClient()
latitude = -37.44609999
longitude = 144.96923385414
max_distance = 1000 #meters    

In [ ]:
location = [144.965234, -37.8076]

In [ ]:
db.collection.drop_indexes()

In [ ]:
for idx in db.collection.list_indexes():
    print (idx)

In [ ]:
db.collection.create_index([( "features.geometry.coordinates", "2dsphere" )])

In [ ]:
## Unwind search

pipeline = [{"$unwind": "$features"},
            {"$match":{'features.geometry.coordinates':[144.965463853978, -37.8077644149636]}},
            {"$group": {"_id": "$features"}}]

pprint.pprint(list(collection.aggregate(pipeline)))

In [ ]:
pipeline2 = [{"$unwind": "$features"},
            {"$match":{'features.properties.Sub Theme' : 'Art Gallery/Museum'}},
            {"$group": {"_id": "$features"}}]

pprint.pprint(list(collection.aggregate(pipeline2)))

## Working code

In [9]:
pipeline = [
        {'$geoNear': {
            'near' : { 'type' : "Point", 'coordinates' : location },
             'distanceField': "dist.calculated",
             'maxDistance': 100,
             'includeLocs': "dist.location",
              'key': 'features.geometry.coordinates',
             'uniqueDocs': False,
             'spherical': True}},
    { "$unwind" : "$features" },
    { "$redact" : {   
        "$cond" : {
            "if" : { "$eq" : [ { "$cmp" : [ "$features.geometry.coordinates", "$dist.location" ] }, 0 ] },
            "then" : "$$KEEP",
            "else" : "$$PRUNE"
        }
    }
}
]

pprint.pprint(list(collection.aggregate(pipeline)))

[{'_id': ObjectId('5ca63a65122b9a6192720138'),
  'dist': {'calculated': 27.27001406965783,
           'location': [144.965463853978, -37.8077644149636]},
  'features': {'geometry': {'coordinates': [144.965463853978,
                                            -37.8077644149636],
                            'type': 'Point'},
               'properties': {'FIELD1': 0,
                              'Feature Name': 'Old Melbourne Gaol Crime & '
                                              'Justice Experience',
                              'Sub Theme': 'Art Gallery/Museum',
                              'Theme': 'Place Of Assembly'},
               'type': 'Feature'},
  'type': 'FeatureCollection'}]


### Check

In [ ]:
a1 = [144.9654, -37.8076]
a2 = [144.9656988, -37.80619915]

distance_finder(a1, a2)

## Implement Automation

In [4]:
click = [144.9607616045045, -37.811441609877804]

In [5]:
location = [144.965234, -37.8076]

In [75]:
pipeline = [
        {'$geoNear': {
            'near' : { 'type' : "Point", 'coordinates' : click },
             'distanceField': "dist.calculated",
             'maxDistance': 500,
             'includeLocs': "dist.location",
              'key': 'features.geometry.coordinates',
             'uniqueDocs': False,
             'spherical': True}},
    { "$unwind" : "$features" },
    { "$redact" : {   
        "$cond" : {
            "if" : { "$eq" : [ { "$cmp" : [ "$features.geometry.coordinates", "$dist.location" ] }, 0 ] },
            "then" : "$$KEEP",
            "else" : "$$PRUNE"
        }
    }
}
]

list(collection.aggregate(pipeline))

[{'_id': ObjectId('5cb0a55fe39a5cbeb4a206d7'),
  'type': 'FeatureCollection',
  'features': {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9602499, -37.81190077]},
   'properties': {'FIELD1': 67,
    'Census year': 2015,
    'Block ID': 74,
    'Property ID': 106018,
    'Base property ID': 106018,
    'Street address': '395-397 Little Lonsdale Street',
    'CLUE small area': 'Melbourne (CBD)',
    'Trading name': 'Horse Bazaar',
    'Industry (ANZSIC4) code': 4520,
    'Industry (ANZSIC4) description': 'Pubs, Taverns and Bars',
    'Seating type': 'Seats - Indoor',
    'Number of seats': 50}},
  'dist': {'calculated': 68.10091452076718,
   'location': [144.9602499, -37.81190077]}},
 {'_id': ObjectId('5cbb067ae39a5cbeb4a82260'),
  'type': 'FeatureCollection',
  'features': {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9615331, -37.81234775]},
   'properties': {'sensor_id': 56,
    'sensor_description': 'Lonsdale St - Elizabeth St (No

In [ ]:
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
pprint.pprint(list(collection.aggregate(pipeline)))
connection.close()

In [5]:
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
finding = collection.find_one( { "_id": ObjectId('5cc683b30e00cbd80dd912ad') })
connection.close()
finding

{'_id': ObjectId('5cc683b30e00cbd80dd912ad'),
 'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9651532, -37.8134944]},
   'properties': {'sensor_id': 1,
    'sensor_description': 'Bourke Street Mall (North)',
    'sensor_name': 'Bou292_T',
    'installation_date': '24/03/2009',
    'status': 'A',
    'note': '',
    'direction_1': 'West',
    'direction_2': 'East'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9651672, -37.81380667]},
   'properties': {'sensor_id': 2,
    'sensor_description': 'Bourke Street Mall (South)',
    'sensor_name': 'Bou283_T',
    'installation_date': '30/03/2009',
    'status': 'A',
    'note': '',
    'direction_1': 'East',
    'direction_2': 'West'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9642949, -37.81101523]},
   'properties': {'sensor_id': 3,
    'sensor_description': 'Melbourne Central',
    'sensor_name': 'Swa295_T',

In [54]:
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
finding = collection.find_one()
connection.close()
finding

{'_id': ObjectId('5ca63a65122b9a6192720138'),
 'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.965463853978, -37.8077644149636]},
   'properties': {'FIELD1': 0,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Old Melbourne Gaol Crime & Justice Experience'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.953478052064, -37.8242574266145]},
   'properties': {'FIELD1': 1,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Polly Woodside'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.967252924593, -37.8266045432686]},
   'properties': {'FIELD1': 2,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Australian Centre for Contemporary Art'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.9688

In [ ]:
from bson import json_util

In [ ]:
def prt(pipeline):
    json_data = list(collection.aggregate(pipeline))
    json_data = json.dumps(json_data, default=json_util.default)
    return json_data

In [11]:
results = list(collection.aggregate(pipeline))

In [ ]:
lis = []
for i in collection.aggregate(pipeline):
    lis.append(i)

In [51]:
results

[{'_id': ObjectId('5cb0a55fe39a5cbeb4a206d7'),
  'type': 'FeatureCollection',
  'features': {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9602499, -37.81190077]},
   'properties': {'FIELD1': 67,
    'Census year': 2015,
    'Block ID': 74,
    'Property ID': 106018,
    'Base property ID': 106018,
    'Street address': '395-397 Little Lonsdale Street',
    'CLUE small area': 'Melbourne (CBD)',
    'Trading name': 'Horse Bazaar',
    'Industry (ANZSIC4) code': 4520,
    'Industry (ANZSIC4) description': 'Pubs, Taverns and Bars',
    'Seating type': 'Seats - Indoor',
    'Number of seats': 50}},
  'dist': {'calculated': 68.10091452076718,
   'location': [144.9602499, -37.81190077]}},
 {'_id': ObjectId('5cbb067ae39a5cbeb4a82260'),
  'type': 'FeatureCollection',
  'features': {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9615331, -37.81234775]},
   'properties': {'sensor_id': 56,
    'sensor_description': 'Lonsdale St - Elizabeth St (No

In [24]:
df = pd.read_csv('data.csv').drop('Open', axis=1)
chart_data = df.to_dict(orient='records')
chart_data = json.dumps(chart_data, indent=2)
data = {'chart_data': chart_data}
data

{'chart_data': '[\n  {\n    "Date": "2009-03-31",\n    "High": 18.79,\n    "Low": 17.78,\n    "Close": 18.37\n  },\n  {\n    "Date": "2009-03-30",\n    "High": 17.76,\n    "Low": 17.27,\n    "Close": 17.48\n  },\n  {\n    "Date": "2009-03-27",\n    "High": 18.62,\n    "Low": 18.05,\n    "Close": 18.13\n  },\n  {\n    "Date": "2009-03-26",\n    "High": 18.88,\n    "Low": 18.12,\n    "Close": 18.83\n  },\n  {\n    "Date": "2009-03-25",\n    "High": 18.31,\n    "Low": 17.52,\n    "Close": 17.88\n  },\n  {\n    "Date": "2009-03-24",\n    "High": 18.21,\n    "Low": 17.84,\n    "Close": 17.93\n  },\n  {\n    "Date": "2009-03-23",\n    "High": 18.59,\n    "Low": 17.31,\n    "Close": 18.33\n  },\n  {\n    "Date": "2009-03-20",\n    "High": 17.65,\n    "Low": 16.88,\n    "Close": 17.06\n  },\n  {\n    "Date": "2009-03-19",\n    "High": 17.45,\n    "Low": 16.92,\n    "Close": 17.14\n  },\n  {\n    "Date": "2009-03-18",\n    "High": 17.22,\n    "Low": 16.6,\n    "Close": 16.96\n  },\n  {\n    "Da

In [4]:
results = {'chart_data': json.loads(json.dumps(json_util.dumps(collection.aggregate(pipeline))))}
results

NameError: name 'collection' is not defined

In [85]:
from flask import jsonify
from random import sample

In [90]:
{'results' : sample(range(1,10), 5)}

{'results': [8, 5, 6, 2, 3]}

In [89]:
jsonify({'results' : sample(range(1,10), 5)})

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.